# Matrix Factorization

In [1]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt

Preprocessing steps can be found [here]() or check the [Collaborative Filtering note]() for the detailed explanation. 

## 1. Loading the data

In [2]:
dir = '../'
# Load the data
with open(dir + 'data/user_to_movie.json', 'rb') as f:
    user_to_movie = pickle.load(f)

with open(dir + 'data/movie_to_user.json', 'rb') as f:
    movie_to_user = pickle.load(f)

with open(dir + 'data/um_to_rating_tr.json', 'rb') as f:
    um_to_rating_tr = pickle.load(f)

with open(dir + 'data/um_to_rating_te.json', 'rb') as f:
    um_to_rating_te = pickle.load(f)

In [3]:
# Count the number of users
N = np.max(list(user_to_movie.keys())) + 1                               # User id starts from 0

# Count the number of movies
m_tr = np.max(list(movie_to_user.keys()))

# Get the maximum movie id from the test set for the movies not in the train set
te_movie_list = [m for (u, m), r in um_to_rating_te.items()]
m_te = np.max(te_movie_list)

M = max(m_tr, m_te) + 1
print("The size of the data: {} users & {} movies".format(N, M))

The size of the data: 1000 users & 200 movies


## 2. Matrix Factorization from scratch

Terms need to be calculated.
- ### Loss fuction J = $\sum_{a, b} (r - \hat{r})^2$ + $||\lambda||$<sub>2</sub>$^2$ 
- ### $\hat{r}$<sub>a, m</sub> = W<sub>a</sub> * U<sub>m</sub> + b<sub>a</sub> + c<sub>m</sub> + $\mu$

where ***b<sub>a</sub>***, ***c<sub>m</sub>*** are the bias terms and ***$\mu$*** is the global average for each user ***a***, movie ***b***. 

In [4]:
# Create the get_loss funtion
def get_loss(x):
    '''
    Calculating the loss
    input: the data in the form of "(user, movie) : rating"
    output: the mean_squared_error
    '''
    n = len(x)
    loss = 0
    for (a, m), r in x.items():
        pred = W[a].dot(U[m]) + b[a] + c[m] + mu
        loss += (r - pred)**2
    return loss/n

In [5]:
# Initialize parameters to update
k = 10                              # latent features

W = np.random.randn(N, k)
U = np.random.randn(M, k)

b = np.zeros(N)
c = np.zeros(M)

mu = np.mean(list(um_to_rating_te.values()))

Since the W and U terms are dependent to each other, ***alternative least square (ALS)*** technique will be applied to get the gradient descent. 

In [6]:
# Train the model
epochs = 5
reg = 20                # regularization penalty

tr_losses = []
te_losses = []

for epoch in range(epochs):

    # Gradient descent on W and b (the parameters related to the user a)
    for a in range(N):

        # Initialize variables for calculation
        term_1 = np.eye(k)*reg
        term_2 = np.zeros(k)
        b_a = 0

        # Updating the parameters along the movie ids while the user id is constant
        for m in user_to_movie[a]:
            r_a_m = um_to_rating_tr[(a, m)]

            term_1 += np.dot(U[m], U[m].T)
            term_2 += (r_a_m - b[a] - c[m] - mu)*U[m]
            b_a += r_a_m - U[m].dot(W[a]) - c[m] - mu

        W[a] = np.linalg.solve(term_1, term_2)
        b[a] = 1/(len(user_to_movie[a])*(1 + reg)) * b_a


    # Gradient descent on U and c (the parameters related to the movie m)
    for m in range(M):

        # Initialize variables for calculation
        term_1 = np.eye(k)*reg
        term_2 = np.zeros(k)
        c_m = 0

        # Updating the parameters along the user ids while the movie id is constant
        try:
            for a in movie_to_user[m]:
                r_a_m = um_to_rating_tr[(a, m)]

                term_1 += np.dot(W[a], W[a].T)
                term_2 += (r_a_m - b[a] - c[m] - mu)*W[a]
                c_m += r_a_m - W[a].dot(U[m]) - b[a] - mu

            U[b] = np.linalg.solve(term_1, term_2)
            c[m] = 1/(len(movie_to_user[m])*(1+reg)) * c_m

        # for the case the movie m has no rating
        except:
            pass

    tr_loss = get_loss(um_to_rating_tr)
    te_loss = get_loss(um_to_rating_te)
    print("{}th epoch train loss : {}, test lss : {}".format(epoch, tr_loss, te_loss))

    tr_losses.append(tr_loss)
    te_losses.append(te_loss)

0th epoch train loss : 3.0192114400891876, test lss : 3.7774942620489265
1th epoch train loss : 2.9784769462496645, test lss : 3.7381509442094645
2th epoch train loss : 3.0100592132613815, test lss : 3.7719534264977774
3th epoch train loss : 3.0089423500456713, test lss : 3.7708839408601
4th epoch train loss : 3.00987620889248, test lss : 3.771876601125883


In [7]:
print("total train loss: ", tr_losses)
print("Total test loss: ", te_losses)

total train loss:  [3.0192114400891876, 2.9784769462496645, 3.0100592132613815, 3.0089423500456713, 3.00987620889248]
Total test loss:  [3.7774942620489265, 3.7381509442094645, 3.7719534264977774, 3.7708839408601, 3.771876601125883]


In [ ]:
# Plot the evaluation
plt.plot(tr_losses, label = "train loss")
plt.plot(te_losses, label = "test loss")
plt.legend()
plt.show()

## 3. A little bit faster way.. 

Instead of looping the user id and movie id repeatedly for each time, we can convert the data into an array so that the computation can speed up. This is using the real power of numpy array. 

Firstly, convert the `user_to_movie` dictionary to contain the rating data in the form of "**user_id : ( the list of movie_ids : an array of rating)**". We'll call this as `user_to_mr`. Also, convert the `movie_to_user` dictionary as a "**movie_id : ( the list of user_ids : an array of rating)**", which will be `movie_to_ur_tr`. The same process will be applied and create `movie_to_ur_te` as follows.

### 3-1. Converting the dictionaries to include the rating data

In [9]:
# Redesigning the user_to_movie and movie_to_user dictionary to contain the rating data
# Create the dictionary in the form of "user_id : (movie_ids, rating_arr)"
user_to_mr = {}
for a, movies in user_to_movie.items():
    rating_list = [um_to_rating_tr[(a, m)] for m in movies]
    rating_arr = np.array(rating_list)
    user_to_mr[a] = (movies, rating_arr)

In [10]:
# Check the result 
print(user_to_mr[1])

([49, 45, 147, 3, 64, 31, 106, 33, 47, 170, 124, 0, 130, 159, 151, 110, 125, 113, 162, 140, 107, 169, 20, 12, 83, 37, 74, 69, 198, 164, 136, 72, 109, 149, 139, 154, 161, 19, 85, 9, 183, 97, 57, 126, 89, 173, 196, 44, 119, 43, 108, 186, 46, 28, 1, 187, 53, 7, 78, 90, 115, 54, 25, 60, 171, 102, 133, 146, 55, 192, 160, 81, 112, 91, 39, 58, 70, 75, 138, 99, 26, 181, 41, 103, 194, 123, 96, 61, 2, 199, 144, 114, 30, 150, 118, 73, 48, 134, 122, 176, 36, 17, 190, 56, 88, 50, 68, 172, 66, 6, 180, 14, 10, 152, 23, 178, 4, 65, 76, 77, 104, 40, 158, 82, 132, 62, 15, 193, 22, 94, 84, 142, 67, 105, 95, 71, 117, 51, 93, 177, 185, 135, 35, 195, 191, 167, 197, 153, 16, 18, 131, 13, 128, 38, 189, 86, 21, 52, 175, 32, 163, 79], array([2.5, 3.5, 4. , 5. , 3. , 4. , 4. , 4.5, 5. , 2.5, 4. , 5. , 4. ,
       3.5, 5. , 3.5, 3. , 5. , 4. , 4. , 4. , 3. , 4. , 4. , 5. , 4.5,
       4.5, 3.5, 4. , 4. , 4.5, 4. , 3. , 4.5, 3.5, 4. , 3.5, 4. , 4.5,
       5. , 4. , 4. , 3. , 5. , 4. , 4. , 5. , 5. , 5. , 5. , 4. 

In [20]:
# Create the dictionary in the form of " movie_id : (user_ids, rating_arr)"
movie_to_ur_tr = {}
for m, users in movie_to_user.items():
    rating_list = [um_to_rating_tr[(a, m)] for a in users]
    rating_arr = np.array(rating_list)
    movie_to_ur_tr[m] = (users, rating_arr)

In [12]:
# Check the result 
print(movie_to_ur_tr[1])

([625, 822, 117, 314, 286, 635, 214, 71, 123, 295, 821, 716, 816, 25, 29, 490, 513, 315, 773, 769, 444, 634, 211, 474, 328, 770, 169, 767, 500, 283, 174, 935, 44, 237, 807, 418, 729, 9, 116, 621, 195, 981, 554, 393, 335, 238, 179, 387, 172, 370, 762, 66, 125, 873, 511, 330, 2, 431, 612, 530, 139, 533, 733, 988, 557, 323, 696, 175, 825, 309, 600, 408, 373, 944, 524, 28, 415, 363, 813, 388, 900, 426, 744, 699, 517, 203, 667, 590, 23, 796, 134, 423, 686, 544, 862, 893, 445, 571, 496, 845, 180, 980, 383, 692, 221, 279, 660, 141, 151, 263, 148, 347, 497, 546, 947, 681, 854, 652, 407, 952, 859, 661, 36, 792, 826, 235, 433, 52, 814, 212, 39, 292, 76, 632, 897, 563, 459, 569, 784, 983, 748, 386, 112, 793, 532, 467, 808, 568, 582, 84, 35, 199, 62, 394, 819, 26, 88, 468, 503, 877, 704, 94, 891, 954, 669, 478, 342, 523, 789, 802, 202, 598, 772, 937, 127, 434, 982, 325, 375, 648, 603, 114, 756, 646, 797, 162, 781, 800, 217, 709, 633, 857, 640, 615, 189, 167, 832, 187, 972, 92, 200, 82, 924, 771, 7

Note that the form of the outcome is the user ids in a list and the ratings in an array. 

In [13]:
# Repeat the same process with the test set
movie_to_ur_te = {}
for (a, m), r in um_to_rating_te.items():
    # If a movie appear for the first time, save the user and rating data as a list
    if m not in movie_to_ur_te:
        movie_to_ur_te[m] = [[a], [r]]
    # If a movie already exist, add the new user id and rating data to the corresponding index
    # Note that the final values will be "movie id : ( a list of user ids , a list of ratings )"
    else:
        movie_to_ur_te[m][0].append(a)
        movie_to_ur_te[m][1].append(r)

# Convert the list of ratings as an array for computation speed later on
for m, (users, ratings) in movie_to_ur_te.items():
    movie_to_ur_te[m][1] = np.array(ratings)

In [14]:
# Check the result 
print(movie_to_ur_te[1])

[[642, 614, 140, 736, 545, 960, 959, 684, 326, 250, 256, 948, 232, 458, 675, 570, 780, 850, 389, 0, 15, 97, 521, 353, 355, 869, 272, 362, 529, 184, 276, 751, 914, 664, 228, 352, 909, 345, 68, 636, 104, 399, 903, 129, 294, 926, 787, 689, 446, 585, 657, 275, 743, 510, 594, 108, 680, 65, 738, 842, 183, 156, 137, 576, 439, 185, 902, 573, 512, 206, 932, 419, 858, 778, 207, 334, 901, 218, 904, 630, 121, 105, 638, 494, 72, 113, 329, 364, 240, 37, 580, 627, 47, 843, 945, 849, 519, 548, 457, 27, 485, 306, 852, 498, 270, 41, 702, 963, 907, 754, 33, 420, 847, 991, 999, 596, 31, 567, 300, 749, 892, 333, 473, 812, 768, 403, 985, 552, 601, 67, 253, 551, 962, 18, 508, 469, 717, 890, 830, 745, 349, 231, 881, 837, 611, 799, 579, 968, 870, 975, 133, 888, 538, 17, 119, 809, 332, 679, 606, 898, 961, 464, 558, 655, 719, 979, 80, 361, 741, 722, 196, 911, 316, 851, 936, 670, 69, 280, 85, 42, 64, 989, 527, 505, 220, 839, 711, 846, 728, 939, 409, 163, 654, 618], array([2.5, 4.5, 2. , 4. , 5. , 5. , 4.5, 0.5, 3

### 3-2. Matrix factorization 

In [22]:
# Create the get_loss funtion
def get_loss(x):
    '''
    Calculating the loss
    input: the data in the form of " movie : ( a list of user ids, an array of ratings )"
    output: the mean_squared_error
    '''
    n = 0
    loss = 0

    for m, (users, ratings) in x.items():
        pred = W[users].dot(U[m]) + b[users] + c[m] + mu
        loss += (ratings - pred).dot(ratings - pred)

        # Count the total number of ratings
        n += len(ratings)

    return loss/n

In [16]:
# Initialize parameters to update
k = 10                              # latent features

W = np.random.randn(N, k)
U = np.random.randn(M, k)

b = np.zeros(N)
c = np.zeros(M)

mu = np.mean(list(um_to_rating_te.values()))

In [23]:
# Train the model
epochs = 10
reg = 0.1                # regularization penalty

tr_losses = []
te_losses = []

for epoch in range(epochs):

    # Gradient descent on W and b (the parameters related to the user a)
    for a in range(N):
        
        # Now we can speed up the computation with the power of numpy
        ###########################################
        movies, rating_arr = user_to_mr[a]
        term_1 = np.eye(k)*reg + np.dot(U[movies].T, U[movies])
        term_2 = (rating_arr - b[a] - c[movies] - mu).dot(U[movies])
        b_a = (rating_arr - U[movies].dot(W[a]) - c[movies] - mu).sum()
        ###########################################

        W[a] = np.linalg.solve(term_1, term_2)
        b[a] = 1/(len(user_to_movie[a])*(1 + reg)) * b_a

    # Gradient descent on U and c (the parameters related to the movie m)
    for m in range(M):
        try:
            ###########################################
            users, rating_arr = movie_to_ur[m]
            term_1 = np.eye(k)*reg + np.dot(W[users].T, W[users])
            term_2 = (rating_arr - b[users] - c[m] - mu).dot(W[users])
            c_m = (rating_arr - W[users].dot(U[m]) - b[users] - mu).sum()
            ###########################################
        
            U[b] = np.linalg.solve(term_1, term_2)
            c[m] = 1/(len(movie_to_user[m])*(1+reg)) * c_m
            
        # for the case the movie m has no rating
        except:
            pass
        
    tr_loss = get_loss(movie_to_ur_tr)
    te_loss = get_loss(movie_to_ur_te)
    print("{}th epoch train loss : {} , test lss : {}".format(epoch, tr_loss, te_loss))

    tr_losses.append(tr_loss)
    te_losses.append(te_loss)

0th epoch train loss : 0.7561272559634299 , test lss : 0.9145395731984773
1th epoch train loss : 0.7561304501153409 , test lss : 0.9145366401795773
2th epoch train loss : 0.7561233597067634 , test lss : 0.9145346220372841
3th epoch train loss : 0.7561235640430944 , test lss : 0.9145323017916246
4th epoch train loss : 0.7561229965832101 , test lss : 0.9145316255733037
5th epoch train loss : 0.7561229945346621 , test lss : 0.9145307213245603
6th epoch train loss : 0.7561229429573093 , test lss : 0.9145305367017733
7th epoch train loss : 0.7561229375917137 , test lss : 0.9145302099297751
8th epoch train loss : 0.7561229324692508 , test lss : 0.9145301557236964
9th epoch train loss : 0.7561229302851759 , test lss : 0.9145300385521866


In [24]:
print("total train loss: ", tr_losses)
print("Total test loss: ", te_losses)

total train loss:  [0.7561272559634299, 0.7561304501153409, 0.7561233597067634, 0.7561235640430944, 0.7561229965832101, 0.7561229945346621, 0.7561229429573093, 0.7561229375917137, 0.7561229324692508, 0.7561229302851759]
Total test loss:  [0.9145395731984773, 0.9145366401795773, 0.9145346220372841, 0.9145323017916246, 0.9145316255733037, 0.9145307213245603, 0.9145305367017733, 0.9145302099297751, 0.9145301557236964, 0.9145300385521866]


In [ ]:
# Plot the evaluation
plt.plot(tr_losses, label = "train loss")
plt.plot(te_losses, label = "test loss")
plt.legend()
plt.show()